###

In [ ]:
# import libraries

import os
import cv2  # OpenCV
import numpy as np



import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, Add, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import numpy as np



In [ ]:
# load and preprocess a map image dataset

dataset_path = "path/to/dataset"

# Define the scale factor for downscaling (e.g., 4x downscale)
scale_factor = 4

# Function to load and preprocess the dataset
def load_and_preprocess_dataset(dataset_path, scale_factor):
    lr_images = []  # Store low-resolution images
    hr_images = []  # Store high-resolution images

    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Load the image
            img = cv2.imread(os.path.join(dataset_path, filename))
            
            # Resize the image to a smaller size for LR
            lr_img = cv2.resize(img, (img.shape[1] // scale_factor, img.shape[0] // scale_factor))

            # Ensure the LR image has the same size as the HR image
            hr_img = cv2.resize(lr_img, (img.shape[1], img.shape[0]))

            # Normalize pixel values to [0, 1]
            lr_img = lr_img / 255.0
            hr_img = hr_img / 255.0

            lr_images.append(lr_img)
            hr_images.append(hr_img)

    return np.array(lr_images), np.array(hr_images)

# Load and preprocess the dataset
lr_images, hr_images = load_and_preprocess_dataset(dataset_path, scale_factor)

# Now you have your LR and HR image pairs for training your SR-GAN


In [ ]:
# Define your generator architecture
def generator_model():
    input_lr = Input(shape=(None, None, 3))
    x = Conv2D(64, 9, padding='same')(input_lr)
    x = LeakyReLU(0.2)(x)

    # Add more Conv2D and BatchNormalization layers as needed

    x = Conv2D(3, 9, padding='same')(x)
    output_hr = Add()([input_lr, x])
    return Model(input_lr, output_hr)

# Define your discriminator architecture
def discriminator_model():
    input_hr = Input(shape=(None, None, 3))
    x = Conv2D(64, 3, padding='same')(input_hr)
    x = LeakyReLU(0.2)(x)

    # Add more Conv2D layers and Dense layers as needed

    output = Conv2D(1, 3, padding='same')(x)
    return Model(input_hr, output)

# Create GAN
def create_gan(generator, discriminator):
    discriminator.trainable = False
    input_lr = Input(shape=(None, None, 3))
    generated_hr = generator(input_lr)
    valid = discriminator(generated_hr)
    return Model(input_lr, [generated_hr, valid])

# Loss functions
mean_squared_error = MeanSquaredError()

def content_loss(hr, generated_hr):
    return mean_squared_error(hr, generated_hr)

def adversarial_loss(valid):
    return mean_squared_error(tf.ones_like(valid), valid)


# Compile and train the GAN
def train_srgan(lr_images, hr_images, epochs, batch_size):
    generator = generator_model()
    discriminator = discriminator_model()
    gan = create_gan(generator, discriminator)

    generator.compile(loss=content_loss, optimizer=Adam(learning_rate=1e-4))
    discriminator.compile(loss=adversarial_loss, optimizer=Adam(learning_rate=1e-4))
    gan.compile(loss=[content_loss, adversarial_loss], loss_weights=[1, 1], optimizer=Adam(learning_rate=1e-4))

    for epoch in range(epochs):
        for batch in range(0, len(lr_images), batch_size):
            # Sample a batch of LR and HR images
            batch_lr = lr_images[batch:batch + batch_size]
            batch_hr = hr_images[batch:batch + batch_size]

            # Generate HR images from LR images
            generated_hr = generator.predict(batch_lr)

            # Train the discriminator
            discriminator.train_on_batch(batch_hr, np.ones((batch_size, 1)))
            discriminator.train_on_batch(generated_hr, np.zeros((batch_size, 1)))

            # Train the generator (SR-GAN)
            gan.train_on_batch(batch_lr, [batch_hr, np.ones((batch_size, 1)])

        # Save the generator model weights periodically

# Testing the SR-GAN
def super_resolve(lr_image, generator):
    return generator.predict(lr_image)

# Load a sample LR map image for testing
# Use your trained generator to enhance the image
# Save the enhanced HR image

if __name__ == "__main__":
    # Train your SR-GAN model
    train_srgan(lr_images, hr_images, epochs=100, batch_size=16)

    # Test the trained model on a sample LR image
    enhanced_hr_image = super_resolve(lr_image, generator)



In [ ]:
# Save the enhanced image to disk
# Save using PIL 


from PIL import Image

# Convert the enhanced HR image from NumPy array to PIL image
enhanced_hr_image_pil = Image.fromarray((enhanced_hr_image * 255).astype('uint8'))

# Define the output file path
output_path = "enhanced_map_image.png"

# Save the enhanced HR image using PIL
enhanced_hr_image_pil.save(output_path)

print(f"Enhanced image saved at: {output_path}")



In [ ]:
# Save the enhanced image to disk
# Save using OpenCV 

import cv2

# Scale the image values back to the range [0, 255] and convert to uint8
enhanced_hr_image_cv2 = (enhanced_hr_image * 255).astype('uint8')

# Define the output file path
output_path = "enhanced_map_image.png"

# Save the enhanced HR image using OpenCV
cv2.imwrite(output_path, cv2.cvtColor(enhanced_hr_image_cv2, cv2.COLOR_RGB2BGR))

print(f"Enhanced image saved at: {output_path}")
